# YAHOO電影爬蟲練習
## 練習爬取電影放映資訊。必須逐步獲取電影的代號、放映地區、放映日期後，再送出查詢給伺服器。

In [12]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd

### 先搜尋全部的電影代號(ID)資訊

In [13]:
# 查看目前上映那些電影，並擷取出其ID資訊
url = 'https://movies.yahoo.com.tw/movie_intheaters.html'
r = requests.get(url)
r.encoding = 'utf-8'
soup=BeautifulSoup(r.text,'lxml')
regex='.*">(\d\d?)<.*'
pattern=re.compile(regex)
regex2='http.*-(.*)'
pattern2=re.compile(regex2)
movies=[]
ids=[]
for movie in soup.find('ul',attrs={'class':'release_list'}).find_all('li'):
    movieName=movie.find('div',attrs={'class':'release_movie_name'})
    movies.append(movieName.a.string.lstrip())
    movieId=re.match(pattern2,movieName.a['href']).group(1)
    ids.append(str(movieId))
    #print(movieName.a.string.lstrip()+'\t'+movieId)
for page in soup.find('div',attrs={'class':'page_numbox'}).find_all('li'):
    result=re.match(pattern,str(page.find('a')))
    if result:
        url_page=url+'?page='+result[1]
        r=requests.get(url_page)
        soup_page=BeautifulSoup(r.text,'lxml')
        for movie in soup_page.find('ul',attrs={'class':'release_list'}).find_all('li'):
            movieName=movie.find('div',attrs={'class':'release_movie_name'})
            movies.append(movieName.a.string.lstrip())
            movieId=re.match(pattern2,movieName.a['href']).group(1)
            ids.append(str(movieId))
            #print(movieName.a.string.lstrip()+'\t'+movieId)
            
dict={
      '電影名稱':movies,
      'ID編號':ids
      }
table=pd.DataFrame(dict)
pd.set_option('display.unicode.ambiguous_as_wide', True)
pd.set_option('display.unicode.east_asian_width', True)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
print(table)

                            電影名稱 ID編號
0                       不完美的正義  10243
1                               姊魅  10403
2                         青春催落去  10404
3           哥哥戀愛了，我怎麼辦！？  10414
4     在黑暗中漫舞：20週年數位修復版  10428
5               巧虎電影：英雄遊樂園  10429
6                     然後我們跳了舞  10451
7                   絕地戰警FOR LIFE  10242
8                           重磅腥聞  10320
9                 魯邦三世 The First  10355
10       動畫電影版 我想吃掉你的胰臟  10377
11                          鬼眼逼人  10379
12                        逃出天堂島  10384
13               Cunningham 機遇之舞  10385
14                        星星知我心  10409
15                            惡靈20  10444
16                        大監獄行動  10450
17                          兔嘲男孩  10283
18                            那一夜  10345
19                        真愛裁會贏  10362
20                          驅魔直播  10375
21                          別告訴她  10382
22                  這一次不再錯過你  10389
23                            菠蘿蜜  10412
24                          霹靂嬌娃  10124
25 

### 指定你有興趣的電影其ID，然後查詢其放映地區資訊。

In [14]:
# 參考前一個步驟中擷取到的ID資訊，並指定ID
movie_id = 9597

In [15]:
url = 'https://movies.yahoo.com.tw/api/v1/areas_by_movie_theater'
payload = {'movie_id':str(movie_id)}

# 模擬一個header
headers = {
    'authority': 'movies.yahoo.com.tw',
    'method': 'GET',
    'path': '/api/v1/areas_by_movie_theater?movie_id=' + str(movie_id),
    'scheme': 'https',
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
    'cookie': 'rxx=9s3x2fws06.1g16irnc&v=1; _ga=GA1.3.2056742944.1551651301; GUC=AQEBAQFczFpdm0IfmwSB&s=AQAAACoo4N5D&g=XMsVBw; BX=4hkdk1decm57t&b=3&s=mr; _ga=GA1.4.2056742944.1551651301; nexagesuid=82843256dd234e8e91aa73f2062f8218; browsed_movie=eyJpdiI6IlJXWWtiSWJaZlNGK2MxQnhscnVUYWc9PSIsInZhbHVlIjoiMXRhMmVHRXRIeUNjc1RBWDJzdGYwbnlIQURmWGsrcjJSMzhkbkcraDNJVUNIZEZsbzU3amlFcVZ1NzlmazJrTGpoMjVrbHk1YmpoRENXaHZTOUw1TmI2ZTZVWHdOejZQZm16RmVuMWlHTTJLaTZLVFZZVkFOMDlTd1wvSGltcytJIiwibWFjIjoiZWQ2ZjA4MmVjZmZlYjlmNjJmYmY2NGMyMDI0Njc0NWViYjVkOWE2NDg0N2RhODMxZjBjZDhiMmJhZTc2MDZhYiJ9; avi=eyJpdiI6Im1NeWFJRlVRWDR1endEcGRGUGJUbVE9PSIsInZhbHVlIjoickRpU3JuUytmcGl6cjF5OW0rNU9iZz09IiwibWFjIjoiY2VmY2NkNzZmM2NhNjY5YzlkOTcyNjE5OGEyMzU0NWYxOTdmMDRkMDY3OWNmMmZjOTMxYjc5MjI5N2Q5NGE5MiJ9; cmp=t=1559391030&j=0; _gid=GA1.4.779543841.1559391031; XSRF-TOKEN=eyJpdiI6IkhpS2hGcDRQaHlmWUJmaHdSS2Q2bHc9PSIsInZhbHVlIjoiOUVoNFk4OHI1UUZmUWRtYXhza0MyWjJSTlhlZ3RnT0VGeVJPN2JuczVRMGRFdWt2OUlsamVKeHRobFwvcHBGM0dhU3VyMXNGTHlsb2dVM2l0U1hpUGxBPT0iLCJtYWMiOiJkZWU4YzJhNjAxMTY3MzE4Y2ExNWIxYmE1ZjE1YWZlZTlhOTcyYjc4M2RlZGY4ZWNjZDYyMTA2NGYwZGViMzc2In0%3D; m_s=eyJpdiI6InpsZHZ2Tk1BZ0dxaHhETml1RjBnUXc9PSIsInZhbHVlIjoiSkNGeHUranRoXC85bDFiaDhySTJqNkJRcWdjWUxjeVRJSHVYZ1wvd2d4bWJZUTUrSHVDM0lUcW5KNHdETFZ4T1lieU81OUhzc1VoUXhZcWk0UDZSQXVFdz09IiwibWFjIjoiYmJkMDJkMDhlODIzMzcyMWY4M2NmYWNjNGVlOWRjMDIwZmVmNzAyMjE3Yzg3ZGY3ODBkZWEzZTI4MTI5ZWNmOSJ9; _gat=1; nexagesd=10',
    'dnt': '1',
    'mv-authorization': '21835b082e15b91a69b3851eec7b31b82ce82afb',
    'referer': 'https://movies.yahoo.com.tw/',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36',
    'x-requested-with': 'XMLHttpRequest',
}
    
resp = requests.get(url, params=payload, headers=headers)
#print(resp.json())  # 若有需要，列印出json原始碼

# 這裡回傳的格式是JSON格式的資料，要解析JSON擷取資料
for p in resp.json():
    print('放映地區: {0}, 代號(area_id): {1}'.format(p['title'], p['area_id']))

放映地區: 台北市, 代號(area_id): 28
放映地區: 新北市, 代號(area_id): 8
放映地區: 桃園, 代號(area_id): 16
放映地區: 新竹, 代號(area_id): 20
放映地區: 苗栗, 代號(area_id): 15
放映地區: 台中, 代號(area_id): 2
放映地區: 南投, 代號(area_id): 13
放映地區: 台南, 代號(area_id): 10
放映地區: 高雄, 代號(area_id): 17
放映地區: 台東, 代號(area_id): 9
放映地區: 澎湖, 代號(area_id): 23


### 指定你想要觀看的放映地區，查詢有上映電影的場次日期

In [16]:
# 指定放映地區
area_id = 28

In [17]:
# 向網站發送請求
url = 'https://movies.yahoo.com.tw/movietime_result.html'
payload = {'movie_id':str(movie_id), 'area_id':str(area_id)}
resp = requests.get(url, params=payload)
resp.encoding = 'utf-8'

soup = BeautifulSoup(resp.text, 'lxml')
movie_date = soup.find_all("label", attrs={'for':re.compile("date_[\d]")})

# 列印播放日期
for date in movie_date:
    print("%s %s" % (date.p.string, date.h3.string))

一月 20
一月 21
一月 22
一月 23
一月 24


### 最後指定觀看的日期，查詢並列印出放映的電影院、放映類型(數位、3D、IMAX 3D...)、放映時間等資訊。

In [18]:
# 選定要觀看的日期
date = "2020-01-21"

In [19]:
# 向網站發送請求，獲取上映的電影院及時間資訊
url = "https://movies.yahoo.com.tw/ajax/pc/get_schedule_by_movie"
payload = {'movie_id':str(movie_id),
           'date':date,
           'area_id':str(area_id),
           'theater_id':'',
           'datetime':'',
           'movie_type_id':''}

resp = requests.get(url, params=payload)
#print(resp.json()['view'])  # 若有需要，列印出json原始碼

soup = BeautifulSoup(resp.json()['view'], 'lxml')
html = soup.find_all("ul", attrs={'data-theater_name':re.compile(".*")})

In [20]:
'''

  試著從上一步驟回傳的電影院資料中，擷取電影院名稱、影片放映類型以及時間表
  
  Your code here.

'''
for theater in html:
    print(theater['data-theater_name'])
    print(theater.find('span',class_='tapR').text)
    for time in theater.find('div',class_='input_picker jq_input_picker').find_all('label'):
        print(time.text)
    print('---------------------------------------------------------')


台北美麗華大直影城
數位
13:20
16:50
---------------------------------------------------------
台北日新威秀影城
數位
14:25
---------------------------------------------------------
喜滿客絕色影城
數位
14:00
17:30
---------------------------------------------------------
台北信義威秀影城
數位
09:50
---------------------------------------------------------
京站威秀影城
數位
11:30
16:50
---------------------------------------------------------
喜樂時代影城南港店
數位
10:20
14:50
---------------------------------------------------------
MUVIE CINEMAS台北松仁威秀
數位
09:45
---------------------------------------------------------
